# Data Pipeline Demo

In [2]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import joblib
import os
import yaml

In [6]:
params_dir = "config/params.yaml"


In [7]:
def load_params(param_dir):
    with open(param_dir, 'r') as file:
        params = yaml.safe_load(file)
        
    return params

In [8]:
params = load_params(params_dir)

In [9]:
params

{'dataset_dir': 'data/raw/',
 'datetime_columns': ['tanggal'],
 'object_columns': ['stasiun', 'critical', 'categori'],
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'predictors': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'range_stasiun': ['DKI1 (Bunderan HI)',
  'DKI2 (Kelapa Gading)',
  'DKI3 (Jagakarsa)',
  'DKI4 (Lubang Buaya)',
  'DKI5 (Kebon Jeruk) Jakarta Barat'],
 'range_pm10': [-1, 800],
 'range_pm25': [-1, 400],
 'range_so2': [-1, 500],
 'range_co': [-1, 100],
 'range_o3': [-1, 160],
 'range_no2': [-1, 100]}

## 1. Data Collection

In [8]:
# fungsi untuk membaca nama file, memuat file, dan menggabungkan dataset
def read_dataset(dataset_dir):
    dataset = pd.DataFrame()

    for i in tqdm(os.listdir(dataset_dir)):
        dataset = pd.concat([pd.read_csv(dataset_dir + i), dataset])
    
    return dataset

In [66]:
# melakukan pembacaan nama file, memuat file, dan menggabungkan dataset
dataset = read_dataset(params["dataset_dir"])

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 272.57it/s]


In [67]:
# cek kondisi dataset
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
150,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
151,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
152,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
153,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [68]:
# terdapat beberapa temuan disini:
# 1. index hanya terlihat sampai 154 padahal jumlah rows sampai 1070
# 2. tanggal hanya terlihat dari bulan 8 dan bulan 9

# harus diselidiki lebih lanjut

In [69]:
# reset index untuk mengatasi poin pertama
dataset.reset_index(inplace = True, drop = True)

In [70]:
# lakukan pengecekan hasil
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1825,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
1826,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
1827,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
1828,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [71]:
# simpan dataset yang telah digabungkan
joblib.dump(dataset, "data/processed/dataset.pkl")

['../data/processed/dataset.pkl']

## 2. Data Definition

In [72]:
# definisikan tipe data, range data serta penjelasan untuk tiap observasi (variabel)

## 3. Data Validation

### 3.1. Tipe Data

In [73]:
# cek tipe data
dataset.dtypes

tanggal     object
stasiun     object
pm10        object
pm25        object
so2         object
co          object
o3          object
no2         object
max         object
critical    object
categori    object
dtype: object

In [74]:
# dari pengecekan data terlihat bahwa semuanya adalah data objek (string), perlu diselidiki lebih lanjut

### 3.2. Range

In [75]:
# pengecekan cakupan data menjadi kacau jika tipe data tidak sesuai
dataset.describe()

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
count,1830,1830,1830,1768,1830,1830,1830,1830,1830,1813,1829
unique,335,5,122,191,121,54,120,86,186,6,4
top,2021-07-15,DKI1 (Bunderan HI),51,81,---,10,24,17,77,PM25,SEDANG
freq,10,366,72,41,114,171,67,90,45,1631,1305


### 3.3. Dimensi Data

In [76]:
# dimensi data kemungkinan besar tidak terpengaruh, namun nanti kita kembali lagi
dataset.shape

(1830, 11)

### 3.4. Handling Columns Error

### 3.5. Handing Column "Tanggal"

In [77]:
# cek tipe data pada kolom tanggal
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tanggal   1830 non-null   object
 1   stasiun   1830 non-null   object
 2   pm10      1830 non-null   object
 3   pm25      1768 non-null   object
 4   so2       1830 non-null   object
 5   co        1830 non-null   object
 6   o3        1830 non-null   object
 7   no2       1830 non-null   object
 8   max       1830 non-null   object
 9   critical  1813 non-null   object
 10  categori  1829 non-null   object
dtypes: object(11)
memory usage: 157.4+ KB


In [78]:
# casting tipe data ke datetime
dataset.tanggal = pd.to_datetime(dataset.tanggal)

In [79]:
# cek perubahan tipe data untuk kolom tanggal
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   object        
 3   pm25      1768 non-null   object        
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 157.4+ KB


### 3.6. Handling Column "PM10"

In [80]:
# terlihat tidak ada masalah pada kolom pm10
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1825,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
1826,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
1827,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
1828,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [81]:
# namun kolom pm10 bukanlah bertipe integer
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   object        
 3   pm25      1768 non-null   object        
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 157.4+ KB


In [82]:
# coba langsung casting ke integer
dataset.pm10 = dataset.pm10.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [83]:
# ternyata ada kategori yang bukan numerik teks, yaitu '---', maka dari itu castingnya gagal
# kita akan melakukan replacing ke data '---' dengan suatu angka numerik, maka dari itu kita harus mencari angka yang unik untuk merepresentasikannya

In [84]:
# angka -1 merupakan nilai yang bisa digunakan karena tidak terdapat di semua kolom dalam dataset kita
# nilai pengganti tidak harus -1, bisa apapun yang penting dapat dengan mudah di bedakan dan unik
dataset[(dataset.eq("-1").any(1)) | (dataset.eq(-1).any(1))]

C:\Users\nvic\AppData\Local\Temp\ipykernel_13764\4056107663.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  dataset[(dataset.eq("-1").any(1)) | (dataset.eq(-1).any(1))]


,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori


In [85]:
# replace data teks dengan -1
# kita tidak bisa replace dengan NaN karena NaN tidak bisa di casting ke integer
dataset.pm10 = dataset.pm10.replace("---", -1).astype(int)

In [86]:
# cek tipe data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1768 non-null   object        
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(9)
memory usage: 150.2+ KB


### 3.7. Handling Column "PM25"

In [87]:
# cek tipe data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1768 non-null   object        
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(9)
memory usage: 150.2+ KB


In [88]:
# coba langsung casting ke integer
dataset.pm25 = dataset.pm25.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [89]:
# juga tidak berhasil selayaknya pm10, ada data yang bukan alphanumerik yaitu '---' sehingga gagal melakukan casting

In [90]:
# terdapat NaN, maka dari itu untuk sementara replace terlebih dahulu dengan -1 seperti data '---'
dataset.pm25.isna().sum()

62

In [91]:
# replacing NaN dengan -1
dataset.pm25.fillna(-1, inplace = True)

In [92]:
# coba lagi untuk casting ke integer
dataset.pm25 = dataset.pm25.replace("---", -1).astype(int)

In [93]:
# cek tipe data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(8)
memory usage: 143.1+ KB


### 3.8. Handling Column "SO2"

In [94]:
# semua yang dilakukan pada pm10 dan pm25 juga diterapkan pada tiap kolom yang seharusnya mempunyai data integer

In [95]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   object        
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(8)
memory usage: 143.1+ KB


In [96]:
dataset.so2 = dataset.so2.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [97]:
dataset.so2 = dataset.so2.replace("---", -1).astype(int)

In [98]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(3), object(7)
memory usage: 135.9+ KB


### 3.9. Handling Column "CO"

In [99]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   object        
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(3), object(7)
memory usage: 135.9+ KB


In [100]:
dataset.co = dataset.co.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [101]:
dataset.co = dataset.co.replace("---", -1).astype(int)

In [102]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(4), object(6)
memory usage: 128.8+ KB


### 3.10. Handling Column "O3"

In [103]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   object        
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(4), object(6)
memory usage: 128.8+ KB


In [104]:
dataset.o3 = dataset.o3.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [105]:
dataset.o3 = dataset.o3.replace("---", -1).astype(int)

In [106]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   int32         
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(5), object(5)
memory usage: 121.6+ KB


### 3.11. Handling Column "NO2"

In [107]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   int32         
 7   no2       1830 non-null   object        
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(5), object(5)
memory usage: 121.6+ KB


In [108]:
dataset.no2 = dataset.no2.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [109]:
dataset.no2 = dataset.no2.replace("---", -1).astype(int)

In [110]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   int32         
 7   no2       1830 non-null   int32         
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(6), object(4)
memory usage: 114.5+ KB


### 3.12. Handling Column "Max"

In [111]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   int32         
 7   no2       1830 non-null   int32         
 8   max       1830 non-null   object        
 9   critical  1813 non-null   object        
 10  categori  1829 non-null   object        
dtypes: datetime64[ns](1), int32(6), object(4)
memory usage: 114.5+ KB


In [112]:
dataset["max"] = dataset["max"].astype(int)

ValueError: invalid literal for int() with base 10: 'PM25'

In [113]:
# untuk kegagalan kali ini disebabkan oleh data yang berbeda dengan sebelumnya, yaitu "PM25" instead of "---"

In [114]:
# perlu kita perhatikan lebih detail
dataset[dataset["max"] == "PM25"]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
1372,2021-12-03,DKI1 (Bunderan HI),49,31,9,19,7,49,PM25,BAIK,NaN


In [115]:
# dari row 1372, dapat kita simpulkan bahwa data kolom max bisa dari pm10 atau no2
# data pada kolom critical adalah PM25, namun itu tidak sesuai dengan definisinya
# maka dari itu kita akan memilih antara PM10 atau NO2
# dan kolom categori, kita dapat berasumsi bahwa nilainya adalah BAIK
# namun dengan salahnya data pada kolom critical (data PM25 namun bukan PM25 yang kritikal, melaikan PM10 dan NO2) maka asumsi kitapun bisa salah
# untuk saat ini kita set dulu ke BAIK untuk nilai di kolom categori, nanti kita liat polanya di EDA untuk categori BAIK
# ingat-ingat saja indexnya adalah 1372

In [116]:
# quick fix the problem
dataset.loc[1372, "max"] = 49
dataset.loc[1372, "critical"] = "PM10"
dataset.loc[1372, "categori"] = "BAIK"

In [117]:
# cek ulang hasilnya
dataset[dataset["max"] == "PM25"]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori


In [118]:
dataset.loc[1372]

tanggal     2021-12-03 00:00:00
stasiun      DKI1 (Bunderan HI)
pm10                         49
pm25                         31
so2                           9
co                           19
o3                            7
no2                          49
max                          49
critical                   PM10
categori                   BAIK
Name: 1372, dtype: object

In [119]:
dataset["max"] = dataset["max"].astype(int)

In [120]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1830 non-null   datetime64[ns]
 1   stasiun   1830 non-null   object        
 2   pm10      1830 non-null   int32         
 3   pm25      1830 non-null   int32         
 4   so2       1830 non-null   int32         
 5   co        1830 non-null   int32         
 6   o3        1830 non-null   int32         
 7   no2       1830 non-null   int32         
 8   max       1830 non-null   int32         
 9   critical  1813 non-null   object        
 10  categori  1830 non-null   object        
dtypes: datetime64[ns](1), int32(7), object(3)
memory usage: 107.4+ KB


### 3.13. Handling Column "Critical"

In [121]:
# cek data unik pada kolom kategorik "critical"
dataset.critical.value_counts()

PM25    1631
PM10      65
O3        57
CO        34
SO2       26
Name: critical, dtype: int64

In [122]:
# semuanya terlihat normal, tidak diperlukan perlakuan khusus

### 3.14. Handling Column "Categori"

In [123]:
# cek data unik untuk kolom kategorik "categori" yang merupakan data label atau dependen variabel
dataset.categori.value_counts()

SEDANG            1305
TIDAK SEHAT        319
BAIK               189
TIDAK ADA DATA      17
Name: categori, dtype: int64

In [124]:
# terdapat data "TIDAK ADA DATA" yang mengindikasikan null value
# bisa kita langsung hapus (drop)

In [125]:
dataset.drop(index = dataset[dataset.categori == "TIDAK ADA DATA"].index, inplace = True)

In [126]:
dataset.categori.value_counts()

SEDANG         1305
TIDAK SEHAT     319
BAIK            189
Name: categori, dtype: int64

In [127]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1813 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1813 non-null   datetime64[ns]
 1   stasiun   1813 non-null   object        
 2   pm10      1813 non-null   int32         
 3   pm25      1813 non-null   int32         
 4   so2       1813 non-null   int32         
 5   co        1813 non-null   int32         
 6   o3        1813 non-null   int32         
 7   no2       1813 non-null   int32         
 8   max       1813 non-null   int32         
 9   critical  1813 non-null   object        
 10  categori  1813 non-null   object        
dtypes: datetime64[ns](1), int32(7), object(3)
memory usage: 120.4+ KB


In [128]:
dataset.categori.value_counts()

SEDANG         1305
TIDAK SEHAT     319
BAIK            189
Name: categori, dtype: int64

In [129]:
# semua tipe data sudah sesuai, bisa kita simpan agar nanti dapat digunakan kembali

In [131]:
joblib.dump(dataset, "data/processed/dataset_clean.pkl")

['../data/processed/dataset_clean.pkl']

## 4. Data Defense

In [132]:
def check_data(input_data, params):
    # check data types
    assert input_data.select_dtypes("datetime").columns.to_list() == params["datetime_columns"], "an error occurs in datetime column(s)."
    assert input_data.select_dtypes("object").columns.to_list() == params["object_columns"], "an error occurs in object column(s)."
    assert input_data.select_dtypes("int").columns.to_list() == params["int32_columns"], "an error occurs in int32 column(s)."

    # check range of data
    assert set(input_data.stasiun).issubset(set(params["range_stasiun"])), "an error occurs in stasiun range."
    assert input_data.pm10.between(params["range_pm10"][0], params["range_pm10"][1]).sum() == len(input_data), "an error occurs in pm10 range."
    assert input_data.pm25.between(params["range_pm25"][0], params["range_pm25"][1]).sum() == len(input_data), "an error occurs in pm25 range."
    assert input_data.so2.between(params["range_so2"][0], params["range_so2"][1]).sum() == len(input_data), "an error occurs in so2 range."
    assert input_data.co.between(params["range_co"][0], params["range_co"][1]).sum() == len(input_data), "an error occurs in co range."
    assert input_data.o3.between(params["range_o3"][0], params["range_o3"][1]).sum() == len(input_data), "an error occurs in o3 range."
    assert input_data.no2.between(params["range_no2"][0], params["range_no2"][1]).sum() == len(input_data), "an error occurs in no2 range."

In [133]:
# jika tidak ada error berarti data sudah sesuai dengan desain kita
check_data(dataset, params)

## 5. Data Splitting

In [134]:
# pisahkan data x dan y (x adalah fitur, y adalah label)
x = dataset[params["predictors"]].copy()
y = dataset.categori.copy()

In [135]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1813 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stasiun  1813 non-null   object
 1   pm10     1813 non-null   int32 
 2   pm25     1813 non-null   int32 
 3   so2      1813 non-null   int32 
 4   co       1813 non-null   int32 
 5   o3       1813 non-null   int32 
 6   no2      1813 non-null   int32 
dtypes: int32(6), object(1)
memory usage: 70.8+ KB


In [136]:
y.value_counts()

SEDANG         1305
TIDAK SEHAT     319
BAIK            189
Name: categori, dtype: int64

In [137]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42, stratify = y)

In [138]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [139]:
joblib.dump(x_train, "data/processed/x_train.pkl")
joblib.dump(y_train, "data/processed/y_train.pkl")
joblib.dump(x_valid, "data/processed/x_valid.pkl")
joblib.dump(y_valid, "data/processed/y_valid.pkl")
joblib.dump(x_test, "data/processed/x_test.pkl")
joblib.dump(y_test, "data/processed/y_test.pkl")

['../data/processed/y_test.pkl']